In [ ]:
# from google.colab import drive

# # Add force_remount=True to fix the error
# drive.mount('/content/drive', force_remount=True)

# # 1. Delete the existing (conflicting) directory
!rm -rf /content/drive

# 2. Try mounting again
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install lightning==2.4.0
# !pip install nemo_toolkit==2.0.0
!pip install transformers datasets sentencepiece librosa jiwer soundfile
# 1. Fix the NumPy version (Crucial for Colab right now)
!pip install "numpy<2.0"

# 2. Install missing NeMo dependency
!pip install hydra-core

# 3. Re-install NeMo with ASR dependencies just to be safe
!pip install "nemo_toolkit[asr]==2.0.0"

# Note: You MUST restart the runtime after running this cell for the NumPy change to take effect.
# Go to "Runtime" -> "Restart Session" (or Ctrl+M .)
!pip uninstall -y numpy
!pip install "numpy<2.0"
!pip install datasets==3.0.2
!pip install pyarrow==14.0.1

In [ ]:
import huggingface_hub
import os
import json
import torch
import glob
import os
import json
import sentencepiece as spm
print("huggingface_hub:", huggingface_hub.__version__)
from omegaconf import OmegaConf, open_dict


from nemo.collections.common.tokenizers import SentencePieceTokenizer
from nemo.collections.asr.models import EncDecCTCModel
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer

print("✅ NeMo imports working fine!")

## Change Path correctly

In [ ]:
BASE_DIR = "Path to your base directory"
MANIFEST_DIR = f"{BASE_DIR}/manifests"
AUDIO_DIR = f"{BASE_DIR}/audio/decoded_audio"
TOKENIZER_DIR = f"{BASE_DIR}/tokenizer_nemo"
CHECKPOINT_DIR = f"{BASE_DIR}/checkpoints"
LOG_DIR = f"{BASE_DIR}/logs"

os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)



TRAIN_MANIFEST = f"{MANIFEST_DIR}/train_manifest.json"
DEV_CLEAN_MANIFEST = f"{MANIFEST_DIR}/dev_clean_manifest.json"
DEV_OTHER_MANIFEST = f"{MANIFEST_DIR}/dev_other_manifest.json"



## Load best checkpoint


In [ ]:
import torch

if not hasattr(torch, '_original_load'):
    def force_unsafe_load(f, map_location=None, pickle_module=None, **pickle_load_args):
        # Force weights_only=False to allow DictConfig objects
        pickle_load_args['weights_only'] = False

        if hasattr(torch, '_original_load'):
            return torch._original_load(f, map_location, pickle_module, **pickle_load_args)
        return torch.jit.load(f) # Fallback

    torch._original_load = torch.load
    torch.load = force_unsafe_load
    print("✅ Applied STRONG patch: 'weights_only=False' is now forced globally.")

✅ Applied STRONG patch: 'weights_only=False' is now forced globally.


In [ ]:
# UPDATE THIS PATH to your actual best checkpoint file
CHECKPOINT_PATH = "PATH to your actual best checkpoint file"

if os.path.exists(CHECKPOINT_PATH):
    print(f"Loading model from: {CHECKPOINT_PATH}")
    # Load model from checkpoint
    asr_model = EncDecCTCModel.load_from_checkpoint(CHECKPOINT_PATH)


    # Set to evaluation mode 
    asr_model.eval()
    asr_model.to("cuda") 
    print("✅ Model loaded and moved to GPU.")
else:
    print(f"❌ Error: Checkpoint not found at {CHECKPOINT_PATH}")

In [ ]:


import json
import soundfile as sf
from jiwer import wer
import re, string

print("\n🔍 Running MANUAL WER evaluation (jiwer 4.x compatible)...")

manifest_path = "Manifest Path"


from omegaconf import open_dict

with open_dict(asr_model.cfg.decoding):
    asr_model.cfg.decoding.strategy = "beam"
    asr_model.cfg.decoding.beam.beam_size = 8
    asr_model.cfg.decoding.beam.alpha = 1.0
    asr_model.cfg.decoding.beam.beta = 0.0

print("✅ Beam decoding enabled for evaluation")


preds = []
refs = []



def detokenize_sentencepiece(text):
    text = text.replace("<s>", "").replace("</s>", "")
    text = text.replace("▁", " ")
    return " ".join(text.split()).strip()

def normalize_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub(r"\s+", " ", text)
    return text.strip()

with open(manifest_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        entry = json.loads(line)
        audio_path = entry["audio_filepath"]
        ref_text = entry["text"].strip()

        audio, sr = sf.read(audio_path)
        if len(audio.shape) > 1:
            audio = audio.mean(axis=1)
        audio = audio.astype("float32")

        raw_pred = asr_model.transcribe([audio])[0]
        clean_pred = detokenize_sentencepiece(raw_pred)

        refs.append(normalize_text(ref_text))
        preds.append(normalize_text(clean_pred))

        if i < 6:
            print("\nREF :", ref_text)
            print("PRED:", clean_pred)
            print("-" * 80)

print("\n⚡ Computing final normalized WER...")
final_wer = wer(refs, preds)

print(f"\n🏆 FINAL NORMALIZED WER = {final_wer * 100:.2f}%\n")

